In [2]:
import torchvision.models as models
import torch 
import onnx
import onnxruntime as ort
import numpy as np
#from openvino.inference_engine import IECore
import timm
import utils

In [5]:
utils.load__analysis('analysisinit', 'mobilenetv3_hpsearch')

In [2]:
INPUT_SHAPE = (3, 224, 224)
batch_size = 1

In [3]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def torch_to_onnx(model, path, batch_size, input_shape):
    dummy_input = torch.randn(batch_size, *input_shape, requires_grad=True)

    torch.onnx.export(model,           # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  path,   # where to save the model (can be a file or file-like object)
                  export_params = True,        # store the trained parameter weights inside the model file
                  opset_version = 10,          # the ONNX version to export the model to
                  do_constant_folding = True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes = {'input' : {0 : 'batch_size'},    # variable length axes
                                 'output' : {0 : 'batch_size'}})
    
    print(f"model saved to {path}")

def load_onnx_model(path):
    model = onnx.load(path)
    onnx.checker.check_model(model)
    return model

In [7]:
torch_model = models.mobilenet_v3_large()
torch_model.eval()

AttributeError: 'MobileNetV3' object has no attribute 'layers'

In [ ]:


path = 'convnext_tiny.onnx'

torch_to_onnx(torch_model, path, batch_size, INPUT_SHAPE)
onnx_model = load_onnx_model(path)

In [5]:
ort_session = ort.InferenceSession(path)

# define model input x to test onx vs torch
inputs = torch.randn(batch_size, *INPUT_SHAPE)

torch_out = torch_model(inputs)

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(inputs)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [6]:
ie = IECore()
device = 'MYRIAD'
model = ie.read_network(model = path)
versions = ie.get_versions(device)
print(versions)

print(f"{device}")
print(f"MKLDNNPlugin version ......... {versions[device].major}.{versions[device].minor}")
print(f"Build ........... {versions[device].build_number}")

{'MYRIAD': <class '__main__.Versions'>}
MYRIAD
MKLDNNPlugin version ......... 2.1
Build ........... 2021.4.0-3839-cd81789d294-releases/2021/4


In [7]:
for input_key in model.input_info:
     print(model.input_info[input_key].input_data.shape)

for input_key in model.input_info:
    input_name = input_key
    model.input_info[input_key].precision = 'FP32'
    break
assert (len(model.input_info.keys()) == 1 or len(model.input_info.keys()) == 2), "Sample supports topologies only with 1 or 2 inputs"


[]


In [8]:
!python "C:\\Program Files (x86)\\Intel\\openvino_2021\\deployment_tools\\model_optimizer\\mo.py" --input_model "C:\\Users\\14135\\Desktop\\birdnet_torch\\convnext_tiny.onnx" --output_dir "C:\\Users\\14135\\Desktop\\birdnet_torch\\" --input_shape (1,3,224,224)

^C


In [ ]:
net = ie.read_network(model = 'convnext_tiny.xml', weights = 'convnext_tiny.bin')

exec_net = ie.load_network(network = net, device_name = device)